<a href="https://colab.research.google.com/github/Girija-Tandon/Tech-Kisan/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv("/content/greenhouse_data1.csv")

In [81]:
df.head(5)

,Temperature (°C),Humidity (%),Moisture (%),Soil Type,Crop Type,Growth Stage,Carbon Emission (ppm),Action
0,21.2,71.9,21.1,Clayey,Crop_91,Flowering,516.04,No Action Needed
1,38.5,62.2,42.5,Saline,Crop_2,Flowering,511.94,No Action Needed
2,32.0,48.6,62.4,Peaty,Crop_13,Vegetative,421.05,Increase Temperature
3,28.0,78.8,53.9,Chalky,Crop_53,Vegetative,433.47,Adjust Lighting
4,14.7,71.1,58.4,Chalky,Crop_32,Flowering,594.41,Adjust Lighting


In [82]:
df['Crop Type'].value_counts()

,count
Crop Type,
Crop_21,12
Crop_70,12
Crop_85,10
Crop_4,10
Crop_3,10
...,...
Crop_96,2
Crop_38,2
Crop_6,2


In [84]:
df.isnull().sum()

,0
Temperature (°C),0
Humidity (%),0
Moisture (%),0
Soil Type,0
Crop Type,0
Growth Stage,0
Carbon Emission (ppm),0
Action,0


In [85]:
df.describe()

,Temperature (°C),Humidity (%),Moisture (%),Soil Type,Crop Type,Growth Stage,Carbon Emission (ppm),Action
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.00000
mean,24.957200,58.915000,41.053400,2.418000,47.336000,1.438000,504.28174,2.08000
std,8.962599,17.129577,17.832168,1.716149,29.106451,1.099351,58.12383,1.47579
min,10.200000,30.300000,10.300000,0.000000,0.000000,0.000000,400.09000,0.00000
25%,17.275000,43.775000,24.450000,1.000000,22.000000,0.000000,453.70500,1.00000
50%,25.400000,58.300000,42.400000,3.000000,45.500000,1.000000,508.93500,2.00000
75%,32.700000,73.550000,56.650000,4.000000,72.000000,2.000000,554.64250,3.00000
max,39.800000,90.000000,70.000000,5.000000,98.000000,3.000000,599.94000,4.00000


In [86]:
df.dtypes

,0
Temperature (°C),float64
Humidity (%),float64
Moisture (%),float64
Soil Type,int64
Crop Type,int64
Growth Stage,int64
Carbon Emission (ppm),float64
Action,int64


In [87]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score


In [98]:
from sklearn.preprocessing import LabelEncoder

# Define categorical columns
categorical_columns = ["Soil Type", "Crop Type", "Growth Stage"]

# Dictionary to store label encoders
label_encoders = {}

# Fit LabelEncoders on categorical columns
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    label_encoders[col].fit(df[col])  # Assuming df is your dataset


In [99]:
# Define features and target variables
X = df.drop(columns=["Carbon Emission (ppm)", "Action"])
y_reg = df["Carbon Emission (ppm)"]  # Target for regression
y_clf = df["Action"]  # Target for classification

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [100]:
df[target_classification] = np.where(df["Moisture (%)"] < 40, "Increase Watering", "Maintain")
df[target_classification] = np.where(df["Temperature (°C)"] > 35, "Turn On Cooling", df[target_classification])


In [101]:
# Split data into training and testing sets
X_train, X_test, y_reg_train, y_reg_test = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)
X_train_clf, X_test_clf, y_clf_train, y_clf_test = train_test_split(X_scaled, y_clf, test_size=0.2, random_state=42)


In [102]:

# Train regression model (Carbon Emission Prediction)
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_reg_train)

RandomForestRegressor(random_state=42)

In [103]:

# Train classification model (Greenhouse Action Prediction)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_clf, y_clf_train)

RandomForestClassifier(random_state=42)

In [104]:
def predict_greenhouse_action(temp, humidity, moisture, soil_type, crop_type, growth_stage):
    # Convert category names to numbers using LabelEncoder
    try:
        soil_encoded = label_encoders["Soil Type"].transform([soil_type])[0]
        crop_encoded = label_encoders["Crop Type"].transform([crop_type])[0]
        growth_encoded = label_encoders["Growth Stage"].transform([growth_stage])[0]
    except ValueError as e:
        return {"error": f"Invalid input: {e}"}

    # Prepare input array
    input_data = np.array([[temp, humidity, moisture, soil_encoded, crop_encoded, growth_encoded]])
    input_scaled = scaler.transform(input_data)

    # Make Predictions
    carbon_pred = regressor.predict(input_scaled)[0]
    action_pred = classifier.predict(input_scaled)[0]

    # Convert prediction back to label
    action_label = label_encoders["Action"].inverse_transform([action_pred])[0]

    return {
        "Predicted Carbon Emission (ppm)": round(carbon_pred, 2),
        "Recommended Greenhouse Action": action_label
    }


In [105]:
print(predict_greenhouse_action(25, 60, 30, "Loamy", "Wheat", "Flowering"))


{'error': "Invalid input: invalid literal for int() with base 10: np.str_('Loamy')"}


In [106]:
def predict_greenhouse_action(temp, humidity, moisture, soil_type, crop_type, growth_stage):
    try:
        # Convert categorical inputs into numerical values
        soil_encoded = label_encoders["Soil Type"].transform([soil_type])[0]
        crop_encoded = label_encoders["Crop Type"].transform([crop_type])[0]
        growth_encoded = label_encoders["Growth Stage"].transform([growth_stage])[0]
    except ValueError as e:
        return {"error": f"Invalid input: {e}"}

    # Prepare input array
    input_data = np.array([[temp, humidity, moisture, soil_encoded, crop_encoded, growth_encoded]])

    # Debug: Print input data before prediction
    print(f"Encoded Input Data: {input_data}")

    # Ensure the scaler was fit properly
    try:
        input_scaled = scaler.transform(input_data)
    except Exception as e:
        return {"error": f"Scaling issue: {e}"}

    # Make Predictions
    carbon_pred = regressor.predict(input_scaled)[0]
    action_pred = classifier.predict(input_scaled)[0]

    # Convert action prediction back to human-readable label
    action_label = label_encoders["Action"].inverse_transform([action_pred])[0]

    return {
        "Predicted Carbon Emission (ppm)": round(carbon_pred, 2),
        "Recommended Greenhouse Action": action_label
    }


In [107]:
print(predict_greenhouse_action(25, 60, 30, "Loamy", "Wheat", "Flowering"))


{'error': "Invalid input: invalid literal for int() with base 10: np.str_('Loamy')"}
